### Importing the required libraries 

In [14]:
import numpy as np
import pandas as pd

In [225]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
# from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import gzip
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
data=pd.read_csv("datacleaned6.csv")

In [39]:
data.head()

,reviewer_id,product_id,reviewer_name,rating,unix_review_time,categories,review_text,pos_feedback,neg_feedback,rating_class,help_prop,review_time,clean_text
0,A30TL5EWN6DFXT,120401325X,christina,4,1400630400,['Cell Phones & Accessories'],Looks Good They look good and stick good! I ju...,0,0,good,0.000000,2014-05-21,look good look good stick good like rounded sh...
1,ASY55RVNIL0UD,120401325X,emily l.,5,1389657600,['Cell Phones & Accessories'],Really great product. These stickers work like...,0,0,good,0.000000,2014-01-14,great product sticker work like review say sti...
2,A2TMXE2AFO7ONB,120401325X,Erica,5,1403740800,['Cell Phones & Accessories'],LOVE LOVE LOVE These are awesome and make my p...,0,0,good,0.000000,2014-06-26,love love love awesome phone look stylish far ...
3,AWJ0WZQYMYFQ4,120401325X,JM,4,1382313600,['Cell Phones & Accessories'],Cute! Item arrived in great time and was in pe...,4,0,good,1.000000,2013-10-21,cute item arrive great time perfect condition ...
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,5,1359849600,['Cell Phones & Accessories'],leopard home button sticker for iphone 4s awes...,2,1,good,0.666667,2013-02-03,leopard home button sticker iphone awesome sta...


In [40]:
data["reviewer_id"].value_counts()

A2NYK9KWFMJV4Y    152
A22CW0ZHY3NJH8    138
A1EVV74UQYVKRY    137
A1ODOGXEYECQQ8    133
A2NOW4U7W3F7RI    132
                 ... 
A1ZQAHY9XQ7CVK      1
A3VDM2I8HU3W00      1
A39P5WRYFYQWEV      1
ATEGLFC1VX8Z        1
AIDOL79XOGWIG       1
Name: reviewer_id, Length: 27874, dtype: int64

In [41]:
data['categories'].nunique()

52

In [42]:
data = data.rename(columns={'reviewer_id': 'userID', 'product_id': 'prod_ID','title': 'prod_name',
                                                 'overall': 'rating' })

In [43]:
data.head()

,userID,prod_ID,reviewer_name,rating,unix_review_time,categories,review_text,pos_feedback,neg_feedback,rating_class,help_prop,review_time,clean_text
0,A30TL5EWN6DFXT,120401325X,christina,4,1400630400,['Cell Phones & Accessories'],Looks Good They look good and stick good! I ju...,0,0,good,0.000000,2014-05-21,look good look good stick good like rounded sh...
1,ASY55RVNIL0UD,120401325X,emily l.,5,1389657600,['Cell Phones & Accessories'],Really great product. These stickers work like...,0,0,good,0.000000,2014-01-14,great product sticker work like review say sti...
2,A2TMXE2AFO7ONB,120401325X,Erica,5,1403740800,['Cell Phones & Accessories'],LOVE LOVE LOVE These are awesome and make my p...,0,0,good,0.000000,2014-06-26,love love love awesome phone look stylish far ...
3,AWJ0WZQYMYFQ4,120401325X,JM,4,1382313600,['Cell Phones & Accessories'],Cute! Item arrived in great time and was in pe...,4,0,good,1.000000,2013-10-21,cute item arrive great time perfect condition ...
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,5,1359849600,['Cell Phones & Accessories'],leopard home button sticker for iphone 4s awes...,2,1,good,0.666667,2013-02-03,leopard home button sticker iphone awesome sta...


In [44]:
data
data.isnull().sum().sum()

15

In [45]:
data=data.dropna()

In [46]:
data.isnull().any().any()

False

In [47]:
print('number of user:',data['userID'].nunique())
print('number of products:',data['prod_ID'].nunique())

number of user: 27872
number of products: 10420


### Creating User item Matrix

In [140]:
# creating user-item matrix
user_item_matrix = df1.pivot_table(index='userID', columns='prod_ID', values='rating', aggfunc='mean')
user_item_matrix.shape

(3090, 4877)

In [141]:
user_item_matrix

prod_ID,6073894996,7887421268,8288853439,8288862993,8288878881,9861203192,9985537742,998554627X,9985612272,B00001W0EQ,...,B00KHADRZS,B00KHV6TIE,B00KJ0K9JI,B00KJLMBQQ,B00KMD91KK,B00KN9UBYI,B00KQT0Q0E,B00KS0BL5A,B00KTUWSSI,B00L3MG3S2
userID,,,,,,,,,,,,,,,,,,,,,
A05793033A673QTUT56XZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A08161909WK3HU7UYTMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100WO06OQR8BQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A104NOZY73BQTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A105RNR6XXJCYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZPZROZJIKBE8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZQGJ5CEAJGXB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZVEABJEV1TQN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Similarity Metrics: 
 Pearson correlation and cosine similarity are the 2 most commonly used similarity metrics in recommendation engines because they capture different aspects of the relationships between users or items in the data.In this, Both Pearson Correlation and Cosine Similarity are used to build recommendation engine built seperately for both to check the performance of teh similarity metrics 
#### 1. Pearsons correlation

In [244]:
p_user_similarity=user_item_matrix.T.corr()
p_user_similarity

userID,A05793033A673QTUT56XZ,A08161909WK3HU7UYTMW,A100WO06OQR8BQ,A104NOZY73BQTQ,A105RNR6XXJCYS,A106Q1A6LGU4DP,A108979C0JSDZS,A108BE4UU6EELV,A108EEYSHGDL6O,A10AAULK3SQEK1,...,AZK7ENEUWO3KK,AZKDLBTHU2BVZ,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZOFNVDHKYKNL,AZPZROZJIKBE8,AZQGJ5CEAJGXB,AZVEABJEV1TQN,AZXFS8GCTSQ5R,AZZYW4YOE1B6E
userID,,,,,,,,,,,,,,,,,,,,,
A05793033A673QTUT56XZ,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A08161909WK3HU7UYTMW,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100WO06OQR8BQ,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A104NOZY73BQTQ,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A105RNR6XXJCYS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZPZROZJIKBE8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
AZQGJ5CEAJGXB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
AZVEABJEV1TQN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


Creating a dataframe that holds 'product id' and 'category'

In [185]:
product_categories=data[['prod_ID','categories']]
product_categories.drop_duplicates(subset='prod_ID',inplace=True)
product_categories

C:\Users\gnvsa\AppData\Local\Temp\ipykernel_17304\2474885230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_categories.drop_duplicates(subset='prod_ID',inplace=True)


,prod_ID,categories
0,120401325X,['Cell Phones & Accessories']
7,3998899561,['Cell Phones & Accessories']
17,6073894996,['Cell Phones & Accessories']
54,7532385086,"['Cell Phones & Accessories', 'Cases, Holsters..."
63,7887421268,"['Cell Phones & Accessories', 'Cases, Holsters..."
...,...,...
190968,B00LH1QZR8,['Cell Phones & Accessories']
190973,B00LH1R3C4,['Cell Phones & Accessories']
190978,B00LH1R5HW,['Cell Phones & Accessories']
190983,B00LH1R6SK,['Cell Phones & Accessories']


##### Recommendation Function

In [245]:
import pandas as pd

def recommend_products_pearson(target_user, n=10):
    if target_user not in user_item_matrix.index or target_user not in p_user_similarity.index:
        print("User ID not found. Please check the User ID.")
        return None

    # Get the similarity scores for the target user with all other users
    similarity_scores = p_user_similarity.loc[target_user]
    
    # Sort the similarity scores and get the indices of the top N similar users
    top_similar_users = similarity_scores.sort_values(ascending=False).head(n).index
    
    # Get the products purchased by the target user
    target_user_purchases = set(user_item_matrix.loc[target_user].dropna().index)
    
    # Find the products purchased by the similar users but not by the target user
    recommended_products = set()
    for user in top_similar_users:
        user_purchases = set(user_item_matrix.loc[user].dropna().index)
        recommended_products.update(user_purchases.difference(target_user_purchases))
        
        if len(recommended_products) >= n:
            break
    
    # Keep only the first num_recommendations products
    recommended_products = list(recommended_products)[:n]
    
    # Convert the list of recommended products to a DataFrame
    recommended_df = pd.DataFrame(recommended_products, columns=['prod_ID'])
    # Merge with the product_categories_df to get the categories
    recommended_df = pd.merge(recommended_df, product_categories, on='prod_ID', how='left')
    
    recommended_df.rename(columns={'prod_ID': 'Recommended Products'},inplace=True)
    
    print('The following are the products recommended for the user:',target_user)
    
    
    return recommended_df

Case 1: Giving both User ID and Number of recommendations as input to the Recommendation function

In [246]:
# Recommend products for user A05793033A673QTUT56XZ
recommend_products_pearson('A05793033A673QTUT56XZ',n=7)

The following are the products recommended for the user: A05793033A673QTUT56XZ


,Recommended Products,categories
0,B009QUDLYW,"['Cell Phones & Accessories', 'Accessories', '..."
1,B009SQSV7G,['Cell Phones & Accessories']
2,B00974L0EA,"['Cell Phones & Accessories', 'Cases, Holsters..."
3,B00B46XSYY,['Cell Phones & Accessories']
4,B00A21L2WM,['Cell Phones & Accessories']
5,B00A21L33K,['Cell Phones & Accessories']
6,B0093CDLM0,"['Cell Phones & Accessories', 'Cases, Holsters..."


Case 2: Giving only User ID as input to the Recommendation function

In [247]:
# Recommend products for user ATX7CZYFXI1KW
recommend_products_pearson('ATX7CZYFXI1KW')

The following are the products recommended for the user: ATX7CZYFXI1KW


,Recommended Products,categories
0,B007406HOC,"['Cell Phones & Accessories', 'Cases, Holsters..."
1,B00CTVJGKI,['Cell Phones & Accessories']
2,B00AK95F56,"['Cell Phones & Accessories', 'Cases, Holsters..."
3,B003ZFL308,"['Cell Phones & Accessories', 'Accessories', '..."
4,B0056LGT44,"['Cell Phones & Accessories', 'Cases, Holsters..."
5,B0044WTQVE,['Cell Phones & Accessories']
6,B002SXRS6G,"['Cell Phones & Accessories', 'Accessories', '..."
7,B004XZHY34,"['Cell Phones & Accessories', 'Accessories', '..."
8,B00DRJTY44,"['Cell Phones & Accessories', 'Cases, Holsters..."
9,B006WVZT3O,"['Cell Phones & Accessories', 'Cases, Holsters..."


Case 3: Giving wrong User ID as input to the Recommendation function

In [248]:
# Recommend products for user User14832 (which is not present in the data)
recommend_products_pearson('User14832')

User ID not found. Please check the User ID.


#### 2. Cosine Similarity

In [234]:
c_user_similarity=cosine_similarity(user_item_matrix.fillna(0))
c_user_similarity=pd.DataFrame(c_user_similarity,index=user_item_matrix.index, columns=user_item_matrix.index)

In [235]:
c_user_similarity.shape

(3090, 3090)

In [238]:
c_user_similarity

userID,A05793033A673QTUT56XZ,A08161909WK3HU7UYTMW,A100WO06OQR8BQ,A104NOZY73BQTQ,A105RNR6XXJCYS,A106Q1A6LGU4DP,A108979C0JSDZS,A108BE4UU6EELV,A108EEYSHGDL6O,A10AAULK3SQEK1,...,AZK7ENEUWO3KK,AZKDLBTHU2BVZ,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZOFNVDHKYKNL,AZPZROZJIKBE8,AZQGJ5CEAJGXB,AZVEABJEV1TQN,AZXFS8GCTSQ5R,AZZYW4YOE1B6E
userID,,,,,,,,,,,,,,,,,,,,,
A05793033A673QTUT56XZ,1.0,0.0,0.0,0.0,0.0,0.121554,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A08161909WK3HU7UYTMW,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A104NOZY73BQTQ,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105RNR6XXJCYS,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZPZROZJIKBE8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AZQGJ5CEAJGXB,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AZVEABJEV1TQN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.169194,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


##### Recommendation Function

In [256]:
import pandas as pd

def recommend_products_cosine(target_user, n=10):
    if target_user not in c_user_similarity.index or target_user not in user_item_matrix.index:
        print("User ID not found. Please check the User ID")
        return None

    # Get the similarity scores for the target user with all other users
    similarity_scores = c_user_similarity.loc[target_user]
    
    # Sort the similarity scores and get the indices of the top N similar users
    top_similar_users = similarity_scores.sort_values(ascending=False).head(n).index
    
    # Get the products purchased by the target user
    target_user_purchases = set(user_item_matrix.loc[target_user].dropna().index)
    
    # Find the products purchased by the similar users but not by the target user
    recommended_products = set()
    for user in top_similar_users:
        user_purchases = set(user_item_matrix.loc[user].dropna().index)
        recommended_products.update(user_purchases.difference(target_user_purchases))
        
        if len(recommended_products) >= n:
            break
    
    # Keep only the first num_recommendations products
    recommended_products = list(recommended_products)[:n]
    # Convert the list of recommended products to a DataFrame
    recommended_df = pd.DataFrame(recommended_products, columns=['prod_ID'])
    # Merge with the product_categories_df to get the categories
    recommended_df = pd.merge(recommended_df, product_categories, on='prod_ID', how='left')
    
    recommended_df.rename(columns={'prod_ID': 'Recommended Products'},inplace=True)
    
    print('The following are the products recommended for the user:',target_user)
    
    return recommended_df

Case 1: Giving both User ID and Number of recommendations as input to the Recommendation function

In [257]:
# Recommend products for user A05793033A673QTUT56XZ
recommend_products_cosine('A05793033A673QTUT56XZ',n=7)

The following are the products recommended for the user: A05793033A673QTUT56XZ


,Recommended Products,categories
0,B002WB2P76,['Cell Phones & Accessories']
1,B0042I3J1I,['Cell Phones & Accessories']
2,B009B3TVX4,"['Cell Phones & Accessories', 'Cases, Holsters..."
3,B0098FKHBU,"['Cell Phones & Accessories', 'Cases, Holsters..."
4,B00CIBSH4K,"['Cell Phones & Accessories', 'Cases, Holsters..."
5,B00AI6K5JW,"['Cell Phones & Accessories', 'Accessories', '..."
6,B00BQ2Y3TA,['Cell Phones & Accessories']


Case 2: Giving only User ID as input to the Recommendation function

In [258]:
# Recommend products for user ATX7CZYFXI1KW
recommend_products_cosine('ATX7CZYFXI1KW')

The following are the products recommended for the user: ATX7CZYFXI1KW


,Recommended Products,categories
0,B004OQOPWU,['Cell Phones & Accessories']
1,B005VNK7T0,"['Cell Phones & Accessories', 'Cases, Holsters..."
2,B0098FKHBU,"['Cell Phones & Accessories', 'Cases, Holsters..."
3,B003Y63FIG,['Cell Phones & Accessories']
4,B0035R2QS4,['Cell Phones & Accessories']
5,B005SUHSN4,"['Cell Phones & Accessories', 'Cases, Holsters..."
6,B003RZTCWC,"['Cell Phones & Accessories', 'Accessories', '..."
7,B005J425JG,"['Cell Phones & Accessories', 'Cases, Holsters..."
8,B00A6WWPAK,"['Cell Phones & Accessories', 'Cases, Holsters..."
9,B004WMDTHS,"['Cell Phones & Accessories', 'Cases, Holsters..."


Case 3: Giving wrong User ID as input to the Recommendation function

In [259]:
recommend_products_cosine('User14832')

User ID not found. Please check the User ID
